In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

import importlib
import rt_analysis as rt
import rt_filter_map_plot

from IPython.display import display, Markdown, Latex, HTML
import json

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

In [2]:
import numpy as np

# Corridor-based metrics for SCCP

* intent is to eventually parameterize this and output formatted report, also methodology may change

In [3]:
date_range = np.arange('2022-04-30', '2022-05-09', dtype='datetime64[D]')

In [4]:
date_range

array(['2022-04-30', '2022-05-01', '2022-05-02', '2022-05-03',
       '2022-05-04', '2022-05-05', '2022-05-06', '2022-05-07',
       '2022-05-08'], dtype='datetime64[D]')

In [5]:
itp_id = 4

In [6]:
corridor = gpd.read_file('./CMCP_StudyAreaBoundary.geojson')

In [7]:
pbar = tqdm()

0it [00:00, ?it/s]

In [8]:
schedule_metrics = []
speed_metrics = []
for date in date_range:
    date = date.astype(dt.date)
    try:
        rt_day = rt_filter_map_plot.from_gcs(itp_id, date, pbar)
        rt_day.add_corridor(corridor)
        _m = rt_filter_map_plot.RtFilterMapper.segment_speed_map(rt_day, corridor=True)
        metrics = rt_day.corridor_metrics()
        schedule_metrics += [metrics['schedule_metric']]
        speed_metrics += [metrics['speed_metric']]
        print(f'complete for date: {date}')
    except:
        print(f'failed for date: {date}')
        continue

found parquet
complete for date: 2022-04-30
found parquet
complete for date: 2022-05-01
found parquet
complete for date: 2022-05-02
found parquet
complete for date: 2022-05-03
found parquet
complete for date: 2022-05-04
found parquet
complete for date: 2022-05-05
found parquet
complete for date: 2022-05-06
found parquet
complete for date: 2022-05-07
found parquet
complete for date: 2022-05-08


In [13]:
np.round(np.mean(schedule_metrics), 0)

462.0

In [14]:
np.round(np.mean(speed_metrics), 0)

4820.0

In [17]:
date_range[4]

numpy.datetime64('2022-05-04')

In [19]:
rt_day = rt_filter_map_plot.from_gcs(itp_id, date_range[4].astype(dt.date), pbar)

found parquet


In [22]:
rt_day

In [23]:
importlib.reload(rt_filter_map_plot)

<module 'rt_filter_map_plot' from '/home/jovyan/data-analyses/rt_delay/rt_filter_map_plot.py'>

In [25]:
rt_day.add_corridor(corridor)

In [39]:
# rt_filter_map_plot.RtFilterMapper.quick_map_corridor(rt_day)

In [27]:
(rt_day.stop_delay_view >> filter(_.corridor)).route_short_name.unique()

array(['801', '73', '10', '41', '45', '1T', '90', '840', '40'],
      dtype=object)

In [35]:
rt_day.set_filter(start_time = '06:00', end_time = '09:00')

In [36]:
_m = rt_filter_map_plot.RtFilterMapper.segment_speed_map(rt_day, corridor=True)

In [40]:
# _m

In [38]:
_m.save('corridor_speedmap.html')